In [ ]:
from __future__ import absolute_import, unicode_literals, print_function
import os
try: os.mkdir('multinest_chains_21cmKAN_ARES_8param')
except OSError: pass

import Global21cmKAN as Global21cmKAN
import numpy as np
import random
import pymultinest
from pymultinest.solve import solve
from matplotlib import pyplot as plt
from matplotlib import rc
from pylinex import Sampler, BurnRule, NLFitter, GaussianLoglikelihood, GaussianModel
from perses.models.KANSignalModelARES import KANSignalModel
#import ares
#from ares.simulations import Global21cm
#from scipy import interpolate
#from distpy.distribution import DistributionSet, DistributionList, UniformDistribution
#from distpy import triangle_plot
#from pylinex.loglikelihood.GaussianLoglikelihoodJoint import GaussianLoglikelihoodJoint
# from perses.models import AresSignalModel, GlobalemuSignalModel
#from perses.models.GlobalemuSignalModelWithUVLF import GlobalemuSignalModelWithUVLF
#from perses.models.AresUVLFModel import AresUVLFModel

rc('figure', figsize=(8.0, 7.0))
plt.rcParams['mathtext.fontset'] = 'cm'

# evaluate a trained instance of 21cmKAN on the given parameter values
emulator = Global21cmKAN.emulator_21cmKAN_ARES.Emulate() # initialize emulator with 21cmKAN data sets
emulator.load_model()  # load pretrained instance of 21cmKAN emulator. Use kwarg 'model_path' to load other models.
#emulator.emulator.summary()
params_list = emulator.par_labels # input physical parameters

# define arrays of test set parameters and signals
params_ARES_test = emulator.par_test.copy() # grab parameter values
signals_ARES_test = emulator.signal_test.copy()

# random_index = np.random.choice(len(params_ARES_test), 1, replace=False)
# par_ARES = np.array(params_ARES_test[random_index][0]) # parameters of mock signal to be fit (i.e., fiducial parameters)
par_ARES = np.array([2.6e39, 0.2, 1e4, 21., 0.05, 2.8e11, 0.49, -0.61]) # signal fit in DJ+23

# true mock ARES signal to be fit (no noise added yet), output by ARES simulation when above parameters are input
T_ARES = np.array([3.31441337e-03,2.18885571e-03,9.69876261e-04,-3.47769970e-04,-1.76936551e-03,-3.30031724e-03,-4.94616668e-03,\
                   -6.71236658e-03,-8.60443097e-03,-1.06277973e-02,-1.27877286e-02,-1.50894416e-02,-1.75377514e-02,-2.01372673e-02,\
                   -2.28920539e-02,-2.58057436e-02,-2.88814461e-02,-3.21216216e-02,-3.55276550e-02,-3.91003115e-02,-8.44328476e-01,\
                   -3.69581203e+00,-6.88031946e+00,-1.03933132e+01,-1.42155707e+01,-1.83334602e+01,-2.27340764e+01,-2.74042637e+01,\
                   -3.23284893e+01,-3.74902613e+01,-4.28714231e+01,-4.84519742e+01,-5.42107445e+01,-6.01247774e+01,-6.61698689e+01,\
                   -7.23211509e+01,-7.85528531e+01,-8.48384646e+01,-9.11513004e+01,-9.74647478e+01,-1.03752457e+02,-1.09988724e+02,\
                   -1.16148600e+02,-1.22207993e+02,-1.28144721e+02,-1.33937733e+02,-1.39567461e+02,-1.45016355e+02,-1.50269156e+02,\
                   -1.55311658e+02,-1.60132725e+02,-1.64721397e+02,-1.69070815e+02,-1.73174817e+02,-1.77029333e+02,-1.80630686e+02,\
                   -1.83978490e+02,-1.87073623e+02,-1.89916909e+02,-1.92510594e+02,-1.94859433e+02,-1.96967595e+02,-1.98840348e+02,\
                   -2.00483914e+02,-2.01904101e+02,-2.03106410e+02,-2.04100919e+02,-2.04891063e+02,-2.05487554e+02,-2.05895372e+02,\
                   -2.06121930e+02,-2.06175300e+02,-2.06062211e+02,-2.05789295e+02,-2.05363287e+02,-2.04791447e+02,-2.04078226e+02,\
                   -2.03230221e+02,-2.02254708e+02,-2.01155044e+02,-1.99937926e+02,-1.98607600e+02,-1.97169935e+02,-1.95627171e+02,\
                   -1.93985915e+02,-1.92249613e+02,-1.90421987e+02,-1.88506639e+02,-1.86508002e+02,-1.84428638e+02,-1.82271965e+02,\
                   -1.80041335e+02,-1.77740027e+02,-1.75371329e+02,-1.72938443e+02,-1.70443046e+02,-1.67888481e+02,-1.65278987e+02,\
                   -1.62616169e+02,-1.59904067e+02,-1.57144185e+02,-1.54340042e+02,-1.51493409e+02,-1.48610951e+02,-1.45692184e+02,\
                   -1.42740961e+02,-1.39760930e+02,-1.36756688e+02,-1.33728768e+02,-1.30681681e+02,-1.27621361e+02,-1.24546615e+02,\
                   -1.21465222e+02,-1.18377934e+02,-1.15288868e+02,-1.12203149e+02,-1.09121496e+02,-1.06051430e+02,-1.02992271e+02,\
                   -9.99516790e+01,-9.69290333e+01,-9.39317288e+01,-9.09594781e+01,-8.80179124e+01,-8.51114508e+01,-8.22398208e+01,\
                   -7.94078096e+01,-7.66174443e+01,-7.38737758e+01,-7.11759347e+01,-6.85276680e+01,-6.59336001e+01,-6.33900067e+01,\
                   -6.09061534e+01,-5.84759605e+01,-5.61085968e+01,-5.37976678e+01,-5.15540860e+01,-4.93673869e+01,-4.72478491e+01,\
                   -4.51857012e+01,-4.31936354e+01,-4.12616718e+01,-3.93905215e+01,-3.75888366e+01,-3.58449059e+01,-3.41654676e+01,\
                   -3.25480086e+01,-3.09887603e+01,-2.94939720e+01,-2.80542477e+01,-2.66756569e+01,-2.53531963e+01,-2.40829107e+01,\
                   -2.28727648e+01,-2.17103464e+01,-2.06034419e+01,-1.95451017e+01,-1.85339409e+01,-1.75774180e+01,-1.66654310e+01,\
                   -1.58007083e+01,-1.49770002e+01,-1.41948406e+01,-1.34534020e+01,-1.27479483e+01,-1.20827886e+01,-1.14493467e+01,\
                   -1.08534802e+01,-1.02871036e+01,-9.75407566e+00,-9.24948703e+00,-8.77361792e+00,-8.32585261e+00,-7.90197137e+00,\
                   -7.50584885e+00,-7.13062459e+00,-6.78002282e+00,-6.44933577e+00,-6.14055552e+00,-5.84934774e+00,-5.57860438e+00,\
                   -5.32383422e+00,-5.08637031e+00,-4.86428648e+00,-4.65791762e+00,-4.46484096e+00,-4.28563440e+00,-4.11899180e+00,\
                   -3.96493747e+00,-3.82161615e+00,-3.68946053e+00,-3.56695801e+00,-3.45502219e+00,-3.35168533e+00,-3.25723441e+00,\
                   -3.17063075e+00,-3.09190848e+00,-3.02046593e+00,-2.95621369e+00,-2.89787484e+00,-2.84605151e+00,-2.79983594e+00,\
                   -2.75939252e+00,-2.72355183e+00,-2.69307938e+00,-2.66706612e+00,-2.64528739e+00,-2.62770427e+00,-2.61884966e+00,\
                   -2.61969814e+00,-2.62420632e+00,-2.63203005e+00,-2.64308846e+00,-2.65727427e+00,-2.67431703e+00,-2.69390212e+00,\
                   -2.71633386e+00,-2.74116587e+00,-2.76815384e+00,-2.79745038e+00,-2.82889134e+00,-2.86230884e+00,-2.89754350e+00,\
                   -2.93461471e+00,-2.97345383e+00,-3.01384831e+00,-3.05590316e+00,-3.09940664e+00,-3.14427957e+00,-3.19061284e+00,\
                   -3.23819785e+00,-3.28698934e+00,-3.33705621e+00,-3.38823912e+00,-3.44055267e+00,-3.49396020e+00,-3.54834631e+00,\
                   -3.60370309e+00,-3.66006137e+00,-3.71732039e+00,-3.77549010e+00,-3.83451913e+00,-3.89442160e+00,-3.95508506e+00,\
                   -4.01659572e+00,-4.07882134e+00,-4.14185842e+00,-4.20556584e+00,-4.27005306e+00,-4.33520373e+00,-4.40104118e+00,\
                   -4.47032868e+00,-4.54222593e+00,-4.61481212e+00,-4.68807223e+00,-4.76197333e+00,-4.83654242e+00,-4.91177537e+00,\
                   -4.98762992e+00,-5.06410452e+00,-5.14117887e+00,-5.21889178e+00,-5.29719856e+00,-5.37609948e+00,-5.45560930e+00,\
                   -5.53568163e+00,-5.61632465e+00,-5.69754808e+00,-5.77935810e+00,-5.86169882e+00,-5.94462838e+00,-6.02808514e+00,\
                   -6.11208572e+00,-6.19666103e+00,-6.28174936e+00,-6.36739091e+00,-6.45354983e+00,-6.54023681e+00,-6.62745942e+00,\
                   -6.71521202e+00,-6.80347471e+00,-6.89226414e+00,-6.98155726e+00,-7.07136018e+00,-7.16166504e+00,-7.25247050e+00,\
                   -7.34380431e+00,-7.43561776e+00,-7.52791446e+00,-7.62043966e+00,-7.71301403e+00,-7.80606522e+00,-7.89959273e+00,\
                   -7.99360004e+00,-8.08810046e+00,-8.18305443e+00,-8.27847608e+00,-8.37437891e+00,-8.47073211e+00,-8.56754597e+00,\
                   -8.66483414e+00,-8.76257282e+00,-8.86076332e+00,-8.95940256e+00,-9.05848630e+00,-9.15801387e+00,-9.25798828e+00,\
                   -9.35839643e+00,-9.45924010e+00,-9.56052356e+00,-9.66224933e+00,-9.76439046e+00,-9.86695242e+00,-9.96993184e+00,\
                   -1.00733387e+01,-1.01771646e+01,-1.02814023e+01,-1.03860444e+01,-1.04910928e+01,-1.05965487e+01,-1.07024137e+01,\
                   -1.08086651e+01,-1.09153068e+01,-1.10223474e+01,-1.11263201e+01,-1.12271461e+01,-1.13283198e+01,-1.14298452e+01,\
                   -1.15317150e+01,-1.16339279e+01,-1.17364752e+01,-1.18393681e+01,-1.19425950e+01,-1.20461527e+01,-1.21500393e+01,\
                   -1.22542476e+01,-1.23587778e+01,-1.24636267e+01,-1.25687950e+01,-1.26742721e+01,-1.27800584e+01,-1.28861506e+01,\
                   -1.29925461e+01,-1.30992404e+01,-1.32062307e+01,-1.33135139e+01,-1.34210899e+01,-1.35289489e+01,-1.36370904e+01,\
                   -1.37455097e+01,-1.38542102e+01,-1.39631817e+01,-1.40724218e+01,-1.41819268e+01,-1.42916891e+01,-1.44017205e+01,\
                   -1.45120005e+01,-1.46225292e+01,-1.47333066e+01,-1.48443366e+01,-1.49555981e+01,-1.50671122e+01,-1.51788462e+01,\
                   -1.52908103e+01,-1.54030141e+01,-1.55154224e+01,-1.56280772e+01,-1.57409217e+01,-1.58540016e+01,-1.59672889e+01,\
                   -1.60807877e+01,-1.61944888e+01,-1.63083700e+01,-1.64224898e+01,-1.65368979e+01,-1.66515909e+01,-1.67662838e+01,\
                   -1.68809768e+01,-1.69960572e+01,-1.71112424e+01,-1.72268233e+01,-1.73424042e+01,-1.74579850e+01,-1.75738588e+01,\
                   -1.76898315e+01,-1.78060100e+01,-1.79206405e+01,-1.80324657e+01,-1.81399194e+01,-1.82457112e+01,-1.83516081e+01,\
                   -1.84576486e+01,-1.85637806e+01,-1.86699859e+01,-1.87763253e+01,-1.88827593e+01,-1.89892853e+01,-1.90958964e+01,\
                   -1.92025764e+01,-1.93093666e+01,-1.94162455e+01,-1.95232030e+01,-1.96302362e+01,-1.97373405e+01,-1.98445014e+01,\
                   -1.99517417e+01,-2.00590499e+01,-2.01664181e+01,-2.02738429e+01,-2.03813208e+01,-2.04888396e+01,-2.05964114e+01,\
                   -2.07040317e+01,-2.08116926e+01,-2.09193909e+01,-2.10271224e+01,-2.11348677e+01,-2.12426552e+01,-2.13504710e+01,\
                   -2.14583079e+01,-2.15661626e+01,-2.16740305e+01,-2.17818796e+01,-2.18897633e+01,-2.19976567e+01,-2.21055517e+01,\
                   -2.22134451e+01,-2.23213337e+01,-2.24292129e+01,-2.25370609e+01,-2.26449158e+01,-2.27527530e+01,-2.28605695e+01,\
                   -2.29683620e+01,-2.30761272e+01,-2.31838594e+01,-2.32915444e+01,-2.33992086e+01,-2.35068329e+01,-2.36144142e+01,\
                   -2.37219489e+01,-2.38294264e+01,-2.39368120e+01,-2.40436858e+01,-2.41468583e+01,-2.42441648e+01,-2.43414169e+01,\
                   -2.44386121e+01])

print('parameters in ARES:', params_list)
print('fiducial parameter values for mock ARES signal being fit (from DJ+23):', par_ARES)
#T_ARES = signals_ARES_test[random_index][0] # mock signal to be fit (no noise added yet)
T_KAN = emulator.predict(par_ARES)  # emulate the mock global 21 cm signal
nu_list = emulator.frequencies
z_list = emulator.redshifts

noise_level = 5 # standard flat noise in mK
true_signal_no_noise = T_ARES.copy()
signal_flat_error = noise_level*np.ones(len(T_ARES))
np.random.seed(2) # set random seed so we get the same error realization for each fit analysis
random_array = np.random.normal(size=len(T_ARES))
gaussian_error = random_array*signal_flat_error
input_true_signal = true_signal_no_noise+gaussian_error

data_signal = input_true_signal
model_signal = KANSignalModel(par_ARES) # initialize KAN signal model emulator, list of params doesn't matter
error_signal = signal_flat_error
gaussian_loglikelihood_signal = GaussianLoglikelihood(data_signal, error_signal, model_signal)
#input_params = np.array([3e-4, 4.2, 1e-6, 0.055, 1.0, 0.1, 10])

vr = 1420.405751
def freq(zs):
    return vr/(zs+1)

def redshift(v):
    return (vr/v)-1

rc('figure', figsize=(7.0, 5.0))
fig, ax = plt.subplots(constrained_layout=True)
ax.minorticks_on()
ax.tick_params(axis='both', which='major', direction = 'out', width = 2, length = 10, labelsize=20)
ax.tick_params(axis='both', which='minor', direction = 'out', width = 2, length = 5, labelsize=20)
ax.set_yticks([50, 0,-50,-100,-150,-200,-250,-300,-350,-400])
ax.set_yticklabels(['50', '0','-50','-100','-150','-200','-250','-300','-350','-400'], fontsize=15, fontname= 'Baskerville')
ax.set_xticks([40, 60, 80, 100, 120, 140, 160, 180, 200, 220])
ax.set_xticklabels(['40', '60', '80', '100', '120', '140', '160', '180', '200', '220'], fontsize=15, fontname= 'Baskerville')
ax.set_ylabel(r'$\delta T_b$ (mK)', fontsize=20, fontname= 'Baskerville')
ax.set_xlabel(r'$\nu$ (MHz)', fontsize=20, fontname= 'Baskerville')
secax = ax.secondary_xaxis('top', functions=(redshift, freq))
secax.tick_params(which='major', direction = 'out', width = 2, length = 10, labelsize=15)
secax.tick_params(which='minor', direction = 'out', width = 1, length = 5, labelsize=15)
secax.set_xlabel(r'$z$', fontsize=20, fontname= 'Baskerville')
secax.set_xticks([5, 10, 15, 20, 30, 50])
secax.set_xticklabels(['5', '10', '15', '20', '30', '50'], fontsize=15, fontname= 'Baskerville')
plt.plot(nu_list, input_true_signal, color='k', alpha=1)
plt.plot(nu_list, T_KAN, color='r', alpha=1, linestyle='--')
plt.plot(nu_list, T_ARES, color='k', alpha=1, linestyle='--')
ax.set_ylim(-300,50)
ax.set_xlim(27.85,236.74)
plt.savefig('ARES_mock_signal_DJ23_5mknoise.png', dpi = 300, bbox_inches='tight', facecolor='w')
plt.show()

##########
# par_ARES = [39.415, 0.2, 4.0, 21.0, -1.301, 11.447158, 0.49, -0.61]
# par_list = ['pop_rad_yield{1}', 'pop_fesc{0}', 'pop_Tmin{0}',  'pop_logN{1}', 'pq_func_par0[0]{0}','pq_func_par1[0]{0}','pq_func_par2[0]{0}','pq_func_par3[0]{0}']
# print('ARES parameter names:', par_list)
# print('fiducial parameter values:', par_ARES)
# parameters_dictionary = ares.util.ParameterBundle('mirocha2017:base')
# updates = {'pop_rad_yield{1}': 10**par_ARES[0], 'pop_fesc{0}': par_ARES[1], 'pop_Tmin{0}': 10**par_ARES[2],
#               'pop_logN{1}': par_ARES[3], 'pq_func_par0[0]{0}': 10**par_ARES[4], 'pq_func_par1[0]{0}': 10**par_ARES[5],
#               'pq_func_par2[0]{0}': par_ARES[6],'pq_func_par3[0]{0}': par_ARES[7]}
# parameters_dictionary.update(updates)
# sim = ares.simulations.Global21cm(**parameters_dictionary, verbose=False, progress_bar=False)
# tau = sim.medium.field.solver.tau
# tau_instance = sim.medium.field.solver.tau_solver
# hmf_instance = sim.pops[0].halos
# sps_instance = sim.pops[0].src
# updates_2 = {'tau_instance': tau_instance, 'hmf_instance': hmf_instance, 'pop_src_instance{0}' : sps_instance}
# parameters_dictionary.update(updates_2)
# sim = ares.simulations.Global21cm(**parameters_dictionary, verbose=False, progress_bar=False)
# sim.run()

# b15 = ares.util.read_lit('bouwens2015')
# b15_Phi = np.array(b15.data['lf'][5.9]['phi'])
# b15_error = np.array(b15.data['lf'][5.9]['err'])
# b15_M_UV = np.array(b15.data['lf'][5.9]['M'])
# magnitude_list = b15_M_UV
# ARES_UVLF = sim.pops[0].LuminosityFunction(5.9, magnitude_list)[1]

# N_UVLF = len(magnitude_list)
# data_UVLF = ARES_UVLF
# model_signal = KANSignalModel(file_path_to_nn='models/emulator.h5')
# model_UVLF = AresUVLFModel(freq(redshifts), magnitude_list, params_list)
# UVLF_noise_level = 2
# error_UVLF = [x*UVLF_noise_level for x in b15_error]
# gaussian_loglikelihood_UVLF = GaussianLoglikelihood(data_UVLF, error_UVLF, model_UVLF)
##########

def priors(cube):
    prior_array = np.array([8, 1, 3.2218, 5, -5.0, 7, 2, -4])*cube + np.array([36, 0, 2.477, 18, 0, 8, 0, 0])
    prior_array[0] = 10**prior_array[0]
    prior_array[2] = 10**prior_array[2]
    prior_array[4] = 10**prior_array[4]
    prior_array[5] = 10**prior_array[5]
    return prior_array

# number of dimensions to fit in nested sampling analysis
n_params = len(params_list)
prefix = "/tutorials/multinest_chains_21cmKAN_ARES_8param/KAN_ARES_8param_nlive1200_tolerance0.1_sampeff0.8_randomsignal_5mKnoise_default17-"
# run MultiNest
kwargs = {'n_live_points':1200, 'evidence_tolerance':0.1,'sampling_efficiency':0.8, 'n_iter_before_update':10}
result = solve(LogLikelihood=gaussian_loglikelihood_signal, Prior=priors, n_dims=n_params, outputfiles_basename=prefix, verbose=True, **kwargs)

print()
print('evidence: %(logZ).1f +- %(logZerr).1f' % result)
print()
print('parameter values:')
for name, col in zip(params_list, result['samples'].transpose()):
    print('%15s : %.3f +- %.3f' % (name, col.mean(), col.std()))

# make marginal plots by running:
# $ python multinest_marginals.py chains/test-
# For that, we need to store the parameter names:
import json
with open('%sparams.json' % prefix, 'w') as f:
    json.dump(params_list, f, indent=2)
